###该代码用于通过k折交叉验证测试模型的精度###
将数据集（lenths.xlsx）读入后进行模型训练并评估
trainModel=False为不需要重新训练，直接从net/last.pt读取模型

In [13]:
import torch
from torch import nn
import torch.nn.functional as F
import readExcel
import time

In [3]:
# Define model
class Net(nn.Module):
    def __init__(self, net_frame):
        super(Net, self).__init__()
        n_input, n_hidden1, n_output = net_frame
        self.net = nn.Sequential(
                    nn.Linear(n_input, n_hidden1),
                    nn.ReLU(),
                    nn.Linear(n_hidden1, n_output),
                    )

    def forward(self, input):
        return self.net(input)

In [14]:
# 设置超参数
learning_rate = 0.0001  # 学习率
net_frame = [7, 3, 6]  # 各层感知机数量
iteration = 150000  # 迭代次数
k_cross=5
trainModel = True

In [15]:
# 准备训练数据
data = readExcel.read_exceldata('A,B,C,D,E,F,G,H,I,J,K,L,M,N', tablePass='lengths.xlsx', print=True)
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      51 non-null     int64  
 1   身高      51 non-null     float64
 2   肩宽      51 non-null     float64
 3   臀宽      51 non-null     float64
 4   大臂      51 non-null     int64  
 5   小臂      51 non-null     int64  
 6   大腿      51 non-null     int64  
 7   小腿      51 non-null     int64  
 8   目测身高    51 non-null     int64  
 9   目测肩宽    51 non-null     int64  
 10  目测大臂长   51 non-null     int64  
 11  目测小臂长   51 non-null     int64  
 12  目测大腿长   51 non-null     int64  
 13  目测小腿长   51 non-null     int64  
dtypes: float64(3), int64(11)
memory usage: 5.7 KB


[[1.0,
  411.0048661512416,
  101.0,
  112.2853507809456,
  57.0,
  48.0,
  84.0,
  78.0,
  174.0,
  47.0,
  27.0,
  24.0,
  41.0,
  37.0],
 [2.0,
  411.0304125000971,
  95.08417323613851,
  107.1167587261676,
  61.0,
  51.0,
  91.0,
  83.0,
  184.0,
  46.0,
  26.0,
  24.0,
  43.0,
  38.0],
 [3.0,
  368.0054347424777,
  93.02150289046077,
  130.0615239031129,
  55.0,
  45.0,
  73.0,
  68.0,
  187.0,
  47.0,
  28.0,
  26.0,
  46.0,
  38.0],
 [4.0,
  417.1726261393478,
  99.02019995940222,
  104.0,
  62.0,
  56.0,
  86.0,
  84.0,
  186.0,
  48.0,
  31.0,
  27.0,
  45.0,
  37.0],
 [5.0,
  386.0634662850138,
  99.04544411531506,
  119.016805536025,
  60.0,
  42.0,
  79.0,
  69.0,
  188.0,
  47.0,
  30.0,
  26.0,
  42.0,
  39.0],
 [6.0,
  408.1482573771448,
  95.13148795220224,
  117.1067888723792,
  62.0,
  54.0,
  91.0,
  83.0,
  174.0,
  45.0,
  27.0,
  23.0,
  40.0,
  32.0],
 [7.0,
  440.0727212632021,
  107.0420478129973,
  114.0,
  62.0,
  52.0,
  95.0,
  81.0,
  162.0,
  40.0,
  30.0

In [17]:
train_x = []
train_y = []
validation_x = []
validation_y = []
loss_acc = 0
lossrate_acc = 0
m = len(data)

In [18]:
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    device=torch.device(f'cuda:{0}')
else:
    device='cpu'
device='cpu'        #在这里由于数据量不大，所以用在与GPU数据传输的时间反而大于CPU上直接训练（数据量不大，但是请求等待总线需要时间）
device

NVIDIA GeForce RTX 2050


'cpu'

In [19]:
#输入模型与数据集，验证准确度
def val_acc(net,val_x,val_y):   #将所有验证集代入模型验证精度，输出各项平均精度
    y=net(val_x)
    # print("prediction:",y)
    error_rate=abs((y-val_y)/val_y)
    # print(type(error_rate))
    print("各项误差:\n",error_rate)
    ave_error_rate=torch.mean(error_rate, dim=0)
    print("平均误差：",ave_error_rate)
    return ave_error_rate

In [9]:
print("数据总数：",str(m))
time_start = time.time()

trainloss_best = []      #记录每一折的训练loss
trainloss_last = []     #清空
val_acc_all=[]
for k in range(k_cross):
    print("第%d折交叉验证" % k)
    train_x = []    #清空
    train_y = []
    validation_x = []
    validation_y = []
    for i in range(m):
        if i % k_cross == k:  # 每5张中有一张放入验证集
            validation_x.append([data[i][1], data[i][2], data[i][3], data[i][4], data[i][5], data[i][6], data[i][7]])
            validation_y.append([data[i][8], data[i][9], data[i][10], data[i][11], data[i][12], data[i][13]])
            # print("已读入验证集" + str(data[i][0]))
        else:
            train_x.append([data[i][1], data[i][2], data[i][3], data[i][4], data[i][5], data[i][6], data[i][7]])
            train_y.append([data[i][8], data[i][9], data[i][10], data[i][11], data[i][12], data[i][13]])
            # print("已读入训练集" + str(data[i][0]))

    # 张量化
    x = torch.tensor(train_x).to(torch.float32).to(device)
    y = torch.tensor(train_y).to(torch.float32).to(device)
    vx = torch.tensor(validation_x).to(torch.float32).to(device)
    vy = torch.tensor(validation_y).to(torch.float32).to(device)

    if trainModel:      #从k折交叉验证数据集重新训练
        net = Net(net_frame).to(device)  # 生成模型对象
        # print(net)

        # 定义训练方法,损失函数,日志间隔
        optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  # .Adam:adam优化器 ,lr:learning_rate 学习率
        loss_func = torch.nn.MSELoss()  # MSELoss损失函数
        log_step_interval = 10000  # 每100轮训练记录一次loss日志用于后期绘图

        # 开始训练
        loss_best = 10000
        lose_last = 0
        best_save = False  # 保存最优模型
        for t in range(iteration):
            # 前向计算->计算损失函数->(从损失函数)反向传播->更新网络
            prediction = net(x)
            loss = loss_func(prediction, y)
            optimizer.zero_grad()  # 清空梯度（可以不写）
            loss.backward()  # 反向传播计算梯度
            optimizer.step()  # 更新网络

            if t % log_step_interval == 0:
                # 控制台输出一下
                # print(str(t)+"Loss = %.8f" % loss.data)
                global_iter_num = t
                print("global_step:{}, loss:{:.8}".format(global_iter_num, loss.item()))
                lose_last = loss.item()
                if lose_last < loss_best:
                    loss_best = lose_last
        print("第%d折训练完成"%k)
        trainloss_best.append(loss_best)
        trainloss_last.append(lose_last)
        #验证集合验证
        val_acc_all.append(val_acc(net,vx,vy))

    else:       #直接调用已训练好模型重新评估
        net = torch.load('net/last.pt')
        pass
time_end = time.time()
print('time cost', time_end - time_start, 's')

数据总数： 51
第0折交叉验证
global_step:0, loss:13280.795
global_step:10000, loss:37.953922
global_step:20000, loss:31.660564
global_step:30000, loss:30.542223
global_step:40000, loss:29.06109
global_step:50000, loss:27.155954
global_step:60000, loss:25.107876
global_step:70000, loss:23.212347
global_step:80000, loss:21.688837
global_step:90000, loss:20.651028
global_step:100000, loss:20.12319
global_step:110000, loss:19.967533
global_step:120000, loss:16.60161
global_step:130000, loss:15.117545
global_step:140000, loss:14.737069
第0折训练完成
各项误差:
 tensor([[5.7490e-03, 1.3999e-02, 3.9292e-02, 2.7068e-02, 2.1650e-02, 4.9186e-02],
        [4.1909e-02, 3.1970e-02, 6.5231e-02, 1.2726e-01, 2.4793e-02, 1.2960e-01],
        [1.5427e-01, 9.7252e-02, 1.1345e-01, 1.5636e-01, 2.6658e-01, 1.6111e-01],
        [1.9228e-02, 1.6304e-02, 3.9157e-02, 1.4971e-02, 7.4545e-02, 8.8789e-02],
        [5.5061e-02, 6.2827e-02, 6.7551e-03, 5.1965e-02, 1.0005e-01, 2.5186e-02],
        [5.8439e-02, 2.9554e-02, 2.1896e-02, 8.979

In [10]:
print("trainloss_best:"+str(trainloss_best))
print("trainloss_best:"+str(trainloss_last))
print("val_acc_all:"+str(val_acc_all))

trainloss_best:[14.737069129943848, 9.895094871520996, 12.830129623413086, 9.976909637451172, 12.107686996459961]
trainloss_best:[14.737069129943848, 9.895094871520996, 12.830129623413086, 9.976909637451172, 12.107686996459961]
val_acc_all:[tensor([0.0444, 0.0511, 0.0506, 0.0659, 0.0831, 0.0698],
       grad_fn=<MeanBackward1>), tensor([0.0579, 0.0759, 0.0524, 0.0926, 0.0851, 0.0921],
       grad_fn=<MeanBackward1>), tensor([0.0704, 0.0688, 0.0815, 0.0576, 0.0963, 0.0710],
       grad_fn=<MeanBackward1>), tensor([0.0605, 0.0869, 0.0933, 0.1020, 0.0897, 0.0589],
       grad_fn=<MeanBackward1>), tensor([0.0515, 0.0364, 0.0578, 0.0237, 0.0536, 0.0425],
       grad_fn=<MeanBackward1>)]


In [11]:
lable=['目测身高','目测肩宽','目测大臂长','目测小臂长','目测大腿长','目测小腿长']
print(lable)
val_acc_all_list=[]
for e in val_acc_all:
    val_acc_all_list.append(e.tolist())
for e in val_acc_all_list:
    print(e)


['目测身高', '目测肩宽', '目测大臂长', '目测小臂长', '目测大腿长', '目测小腿长']
[0.044388823211193085, 0.051118072122335434, 0.0505562424659729, 0.06586953997612, 0.0831184908747673, 0.06978590041399002]
[0.05790172144770622, 0.07589936256408691, 0.052368033677339554, 0.09259636700153351, 0.08508194237947464, 0.09214834868907928]
[0.07044940441846848, 0.06881637871265411, 0.08147630840539932, 0.057625912129879, 0.09634065628051758, 0.07096338272094727]
[0.06049622967839241, 0.08694088459014893, 0.09329331666231155, 0.10200075060129166, 0.08972788602113724, 0.05894298478960991]
[0.05147934705018997, 0.036426037549972534, 0.05779798701405525, 0.02367136999964714, 0.0535852424800396, 0.0425269640982151]


In [12]:
lable=['目测身高','目测肩宽','目测大臂长','目测小臂长','目测大腿长','目测小腿长']
print(lable,'总平均误差')
print(torch.mean(torch.tensor(val_acc_all_list),dim=0).tolist())

['目测身高', '目测肩宽', '目测大臂长', '目测小臂长', '目测大腿长', '目测小腿长'] 总平均误差
[0.05694310739636421, 0.06384014338254929, 0.06709837913513184, 0.06835278123617172, 0.08157084137201309, 0.06687351316213608]
